# Créations des tables populations et countries à partir d'un ficher .csv et .txt

In [1]:
import pandas as pd
import numpy as np

## TABLE POPULATIONS

In [2]:
populations = pd.read_csv('breaf5-populations.csv')

In [3]:
populations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16060 entries, 0 to 16059
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   LOCATION    16060 non-null  object 
 1   INDICATOR   16060 non-null  object 
 2   SUBJECT     16060 non-null  object 
 3   MEASURE     16060 non-null  object 
 4   FREQUENCY   16060 non-null  object 
 5   TIME        16060 non-null  int64  
 6   Value       16060 non-null  float64
 7   Flag Codes  0 non-null      float64
dtypes: float64(2), int64(1), object(5)
memory usage: 1003.9+ KB


In [4]:
len(populations.columns)

8

In [5]:
populations.SUBJECT.unique()

array(['TOT', 'MEN', 'WOMEN'], dtype=object)

Les valeurs de la colonne SUBJECT sont transformés en valeur numérique pour pouvoir les supprimer par la suite (ne semble pas fonctionner avec des valeurs de type str). Cela permet de supprimer les lignes ne recensant qu'une partie de la population.

In [6]:
populations.SUBJECT.replace(['TOT', 'MEN', 'WOMEN'], [1,2,3], inplace=True)

In [7]:
populations.drop(populations[(populations.SUBJECT > 1)].index, inplace=True)

In [8]:
populations.SUBJECT.unique()

array([1], dtype=int64)

In [9]:
populations.MEASURE.unique()

array(['AGRWTH', 'MLN_PER'], dtype=object)

On fait la même avec la colonne MEASURE pour supprimer ls lignes ayant la valeur AGRWTH.

In [10]:
populations.MEASURE.replace(['MLN_PER', 'AGRWTH'], [1,2], inplace=True)

In [11]:
populations.drop(populations[(populations.MEASURE > 1)].index, inplace=True)

In [12]:
populations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4029 entries, 143 to 15987
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   LOCATION    4029 non-null   object 
 1   INDICATOR   4029 non-null   object 
 2   SUBJECT     4029 non-null   int64  
 3   MEASURE     4029 non-null   int64  
 4   FREQUENCY   4029 non-null   object 
 5   TIME        4029 non-null   int64  
 6   Value       4029 non-null   float64
 7   Flag Codes  0 non-null      float64
dtypes: float64(2), int64(3), object(3)
memory usage: 283.3+ KB


Multiplication des valeurs par un million pour les mettre à l'échelle.

In [13]:
populations.Value = round(populations.Value * 1000000, 0)

In [14]:
populations.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
143,AUS,POP,1,1,A,1950,8178700.0,NaN
144,AUS,POP,1,1,A,1951,8421700.0,NaN
145,AUS,POP,1,1,A,1952,8636500.0,NaN
146,AUS,POP,1,1,A,1953,8815300.0,NaN
147,AUS,POP,1,1,A,1954,8986500.0,NaN


Création d'un ficher csv ne contenant que les lignes et colonnes d'intérêt pour le breaf.

In [15]:
populations[['LOCATION', 'TIME', 'Value']].to_csv('breaf5-populations_drop', index=False, encoding='utf-8')

In [16]:
populations_drop = pd.read_csv('breaf5-populations_drop', index_col=False)
populations_drop.head()

,LOCATION,TIME,Value
0,AUS,1950,8178700.0
1,AUS,1951,8421700.0
2,AUS,1952,8636500.0
3,AUS,1953,8815300.0
4,AUS,1954,8986500.0


In [17]:
populations_drop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4029 entries, 0 to 4028
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   LOCATION  4029 non-null   object 
 1   TIME      4029 non-null   int64  
 2   Value     4029 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 94.6+ KB


Envoie dans la base de donnée déjà créé à l'avance 'CountriesDatas' dans PHPmyadmin. Pour cela, une table 'populations' est créée avec 4 colonnes dont un index auto-incrémenté en clé primaire, puis les données sont envoyés dans la table nouvellement créée 'populations'.

In [18]:
import mysql.connector as mysql
from mysql.connector import Error

user = 'root'
password = 'example'
host = 'localhost'
port = '3308'
database = 'CountriesDatas'

try:
    conn = mysql.connect(host=host, port=port, user=user, password=password, database=database)
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        cursor.execute('DROP TABLE IF EXISTS populations;')
        print('Creating table....')
        cursor.execute("CREATE TABLE populations(id INT PRIMARY KEY NOT NULL AUTO_INCREMENT, LOCATION VARCHAR(10) NOT NULL, TIME year NOT NULL, Value BIGINT NOT NULL);")
        print("Table is created....")
        for i,row in populations_drop.iterrows():
            sql = "INSERT INTO populations (LOCATION, TIME, Value) VALUES (%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            print("Record inserted")
            conn.commit()
        cursor.close()
        conn.close()
except Error as e:
            print("Error while connecting to MySQL", e)

You're connected to database:  ('countriesdatas',)
Creating table....
Table is created....
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record insert

## TABLE COUNTRIES

In [19]:
countries = pd.read_fwf('breaf5-country.txt')
countries

,colonne
0,ABW __****__ Aruba
1,AFG __****__ Afghanistan
2,AFRI __****__ Africa
3,AGO __****__ Angola
4,AIA __****__ Anguilla
...,...
227,WSM __****__ Samoa
228,YEM __****__ Yemen
229,ZAF __****__ South Africa
230,ZMB __****__ Zambia


Création de deux nouvelles colonnes pour accueillir les données extraites des données d'origine (j'avoue, j'ai triché, j'ai rajouté manuellement une ligne au fichier .txt, car la première ligne devient le titre de la table).

In [20]:
countries['code'] = 0
countries['name'] = 0

Les deux boucles suivantes permettent d'extraire les données correspondant à 'code' et 'name' grâce à des regex.

In [21]:
for i in countries:
    countries.code = countries.colonne.str.extract('([A-Z]+)\s')

In [22]:
for i in countries:
    countries.name = countries.colonne.str.extract('\s([A-Za-z\s]+)')

In [23]:
countries

,colonne,code,name
0,ABW __****__ Aruba,ABW,Aruba
1,AFG __****__ Afghanistan,AFG,Afghanistan
2,AFRI __****__ Africa,AFRI,Africa
3,AGO __****__ Angola,AGO,Angola
4,AIA __****__ Anguilla,AIA,Anguilla
...,...,...,...
227,WSM __****__ Samoa,WSM,Samoa
228,YEM __****__ Yemen,YEM,Yemen
229,ZAF __****__ South Africa,ZAF,South Africa
230,ZMB __****__ Zambia,ZMB,Zambia


In [24]:
countries[['code', 'name']].to_csv('breaf5-countries_drop', index=False, encoding='utf-8')

In [25]:
countries_drop = pd.read_csv('breaf5-countries_drop', index_col=False)
countries_drop.head()

,code,name
0,ABW,Aruba
1,AFG,Afghanistan
2,AFRI,Africa
3,AGO,Angola
4,AIA,Anguilla


In [26]:
import mysql.connector as mysql
from mysql.connector import Error

user = 'root'
password = 'example'
host = 'localhost'
port = '3308'
database = 'CountriesDatas'

try:
    conn = mysql.connect(host=host, port=port, user=user, password=password, database=database)
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        cursor.execute('DROP TABLE IF EXISTS countries;')
        print('Creating table....')
        cursor.execute("CREATE TABLE countries(code VARCHAR(10) PRIMARY KEY NOT NULL, name VARCHAR(50) NOT NULL);")
        print("Table is created....")
        for i,row in countries_drop.iterrows():
            sql = "INSERT INTO countries VALUES (%s,%s)"
            cursor.execute(sql, tuple(row))
            print("Record inserted")
            conn.commit()
            cursor.close()
            conn.close()
except Error as e:
            print("Error while connecting to MySQL", e)

You're connected to database:  ('countriesdatas',)
Creating table....
Table is created....
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record insert

# JOIN de la table 'populations' avec la table 'countries'.

In [27]:
try:
    conn = mysql.connect(host=host, port=port, user=user, password=password, database=database)
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        cursor.execute(f"""SELECT populations.id, countries.name, populations.TIME, populations.Value FROM populations JOIN countries 
                            ON populations.LOCATION = countries.code""")
        population_country = cursor.fetchall()
        cursor.close()
        conn.close()
        
except Error as e:
            print("Error while connecting to MySQL", e)

You're connected to database:  ('countriesdatas',)


In [28]:
for i in population_country:
    print(i)

(1, 'Australia', 1950, 8178700)
(2, 'Australia', 1951, 8421700)
(3, 'Australia', 1952, 8636500)
(4, 'Australia', 1953, 8815300)
(5, 'Australia', 1954, 8986500)
(6, 'Australia', 1955, 9199700)
(7, 'Australia', 1956, 9425500)
(8, 'Australia', 1957, 9640200)
(9, 'Australia', 1958, 9842400)
(10, 'Australia', 1959, 10056400)
(11, 'Australia', 1960, 10275000)
(12, 'Australia', 1961, 10508200)
(13, 'Australia', 1962, 10700500)
(14, 'Australia', 1963, 10906900)
(15, 'Australia', 1964, 11121600)
(16, 'Australia', 1965, 11340900)
(17, 'Australia', 1966, 11599498)
(18, 'Australia', 1967, 11799078)
(19, 'Australia', 1968, 12008635)
(20, 'Australia', 1969, 12263014)
(21, 'Australia', 1970, 12507349)
(22, 'Australia', 1971, 13067265)
(23, 'Australia', 1972, 13303664)
(24, 'Australia', 1973, 13504538)
(25, 'Australia', 1974, 13722571)
(26, 'Australia', 1975, 13892995)
(27, 'Australia', 1976, 14033083)
(28, 'Australia', 1977, 14192234)
(29, 'Australia', 1978, 14359255)
(30, 'Australia', 1979, 14515729

# Conversion du résultat en dataframe

In [29]:
dataframe = pd.DataFrame(population_country, columns = ['id', 'countrie', 'year', 'population'])

# Bonus : exportation en format .csv

In [30]:
dataframe.to_csv('breaf5-dataframe', index=False, encoding='utf-8')

In [31]:
data_breaf = pd.read_csv('breaf5-dataframe', index_col=False)
data_breaf.head()

,id,countrie,year,population
0,1,Australia,1950,8178700
1,2,Australia,1951,8421700
2,3,Australia,1952,8636500
3,4,Australia,1953,8815300
4,5,Australia,1954,8986500
